In [12]:
import numpy
import h5py
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from sklearn.cross_validation import StratifiedKFold
from keras.layers import Dropout
from keras.regularizers import l2, activity_l2
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import Convolution2D
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.cross_validation import train_test_split

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
import numpy
import tensorflow as tf
tf.python.control_flow_ops = tf
import scipy.misc # to visualize only
x = numpy.fromfile('dataset/train_x.bin', dtype='uint8')
x = x.reshape((100000,60,60))
x[x<255] = 0

y = pd.read_csv('dataset/train_y.csv')
y=y['Prediction']
y=y.as_matrix()

#scipy.misc.imshow(x[10]) # to visualize only

In [14]:
x = x[95000:]
y = y[95000:]

In [15]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Convolution2D(64, 3, 3, border_mode='valid', input_shape=(1, 60, 60), activation='relu'))#58
	model.add(Convolution2D(64, 3, 3, activation='relu'))#56
	model.add(MaxPooling2D(pool_size=(2, 2)))#28
	model.add(Convolution2D(64, 3, 3, activation='relu'))#26
	model.add(Convolution2D(64, 3, 3, activation='relu'))#24
	model.add(MaxPooling2D(pool_size=(2, 2)))#12
	model.add(Convolution2D(64, 3, 3, activation='relu'))#10
	model.add(Convolution2D(64, 3, 3, activation='relu'))#8
	model.add(MaxPooling2D(pool_size=(2, 2)))#4
	model.add(Convolution2D(64, 3, 3, activation='relu'))#2
	model.add(MaxPooling2D(pool_size=(2, 2)))#1
	model.add(Dropout(0.3))
	model.add(Flatten())
	model.add(Dense(128, activation='relu',W_regularizer=l2(0.01)))
	model.add(Dense(50, activation='relu',W_regularizer=l2(0.01)))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [ ]:
model = larger_model()
model.load_weights("weights4_layers_updated_11layer_9700_wts.h5",by_name=False)
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))
model.save("weights4_layers_updated_11layer_part3.h5")

## Performing 3 fold CV

In [16]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3)
#StratifiedKFold(n_splits=2, random_state=None, shuffle=False)
for i in range(5):
    print "testing for epoch"
    print i
    for train_index, test_index in skf.split(x, y):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        num_pixels = X_train.shape[1] * X_train.shape[2]
        X_train = X_train.reshape(X_train.shape[0], 1,60,60).astype('float32')
        X_test = X_test.reshape(X_test.shape[0], 1,60,60).astype('float32')
        X_train = X_train / 255
        X_test = X_test / 255
        y_train = np_utils.to_categorical(y_train)
        y_test = np_utils.to_categorical(y_test)
        num_classes = y_test.shape[1]
        print X_train.shape
        print X_test.shape
        print y_train.shape
        print y_test.shape
        print num_classes

        model = larger_model()
        #model.load_weights("weights4_layers_updated_11layer_9700_wts.h5",by_name=False)
        # Fit the model
        model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=i, batch_size=200, verbose=2)
        # Final evaluation of the model
        scores = model.evaluate(X_test, y_test, verbose=0)
        print("Baseline Error: %.2f%%" % (100-scores[1]*100))
    #model.save("weights4_layers_updated_11layer_part3.h5")


(3326, 1, 60, 60)
(1674, 1, 60, 60)
(3326, 19)
(1674, 19)
19
Train on 3326 samples, validate on 1674 samples
Baseline Error: 96.95%
(3334, 1, 60, 60)
(1666, 1, 60, 60)
(3334, 19)
(1666, 19)
19
Train on 3334 samples, validate on 1666 samples
Baseline Error: 98.08%
(3340, 1, 60, 60)
(1660, 1, 60, 60)
(3340, 19)
(1660, 19)
19
Train on 3340 samples, validate on 1660 samples
Baseline Error: 96.81%
(3326, 1, 60, 60)
(1674, 1, 60, 60)
(3326, 19)
(1674, 19)
19
Train on 3326 samples, validate on 1674 samples
Epoch 1/1
71s - loss: 4.3180 - acc: 0.0755 - val_loss: 2.8356 - val_acc: 0.0962
Baseline Error: 90.38%
(3334, 1, 60, 60)
(1666, 1, 60, 60)
(3334, 19)
(1666, 19)
19
Train on 3334 samples, validate on 1666 samples
Epoch 1/1
71s - loss: 4.3066 - acc: 0.0714 - val_loss: 2.8668 - val_acc: 0.0990
Baseline Error: 90.10%
(3340, 1, 60, 60)
(1660, 1, 60, 60)
(3340, 19)
(1660, 19)
19
Train on 3340 samples, validate on 1660 samples
Epoch 1/1


KeyboardInterrupt: 